In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.utils.data import Dataset, DataLoader, TensorDataset


In [18]:
class Conv1DNet(nn.Module):

    def __init__(self, n_chan=64, kernel_size=3, in_ch=1, out_ch=1, bias=True):
        super(Conv1DNet, self).__init__()
        
        # Expected input size: [Batch_size, in_ch,  Time Steps (180)]
        first_layer = [nn.Conv1d(in_channels=in_ch, out_channels=n_chan, kernel_size=kernel_size, bias=bias, padding = kernel_size//2),
                       nn.MaxPool1d(3),
                       nn.ReLU(True)]
        self.first_layer = nn.Sequential(*first_layer)
        
        feature_block = []
        for _ in range(1):
            feature_block += [nn.Conv1d(in_channels=n_chan, out_channels=n_chan, kernel_size=kernel_size, bias=bias, padding = kernel_size//2),
                              nn.MaxPool1d(3),
                              nn.ReLU(True)]
        self.feature_block = nn.Sequential(*feature_block)
        
        # final layer - reduced the input to [Batch_size, n_chan, 20]
        final_layer = [nn.Linear(20*n_chan, 2)]
        self.final_layer = nn.Sequential(*final_layer)

    def forward(self, input_tensor):
        out = self.first_layer(input_tensor)
        out = self.feature_block(out)
        out = out.reshape(out.shape[0], -1)
        out = self.final_layer(out)
        return out


In [62]:
# TBD - Input/target from Tom's preprocessing, divide to train/val/test
inputs = torch.randn([20,1,180]) 
targets = torch.randint(0,2,[20])
val_inputs = torch.randn([20,1,180])
val_targets = torch.randint(0,2,[20])
trainset = TensorDataset(inputs, targets)
valset = TensorDataset(val_inputs, val_targets)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=True)


In [82]:
net = Conv1DNet()
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), lr=5e-4)
batch_size = 16
loss_history = []
accuracy_history = []
for epoch in range(10):
    for batch_ndx, sample in enumerate(train_loader):
        opt.zero_grad()
        out = net(sample[0])
        loss = loss_fn(out, sample[1])
        loss.backward()
        opt.step()
        loss_history.append(loss.detach().item())
    with torch.no_grad():
        total = 0
        correct = 0
        for batch_ndx, sample in enumerate(val_loader):
            out = net(sample[0])
            _, predicted = torch.max(out.data, 1)
            total += sample[1].size(0)
            correct += (predicted == sample[1]).sum().item()
        print('epoch %2d, accuracy %.2f' % (epoch+1, correct/total))
        accuracy_history.append(correct/total)

epoch  1, accuracy 0.55
epoch  2, accuracy 0.55
epoch  3, accuracy 0.55
epoch  4, accuracy 0.55
epoch  5, accuracy 0.55
epoch  6, accuracy 0.55
epoch  7, accuracy 0.40
epoch  8, accuracy 0.55
epoch  9, accuracy 0.45
epoch 10, accuracy 0.50
